# Converting matlab data into python

Use this Jupyter notebook to convert Expo data from .mat to .npy files

In [1]:
import numpy as np
import math, os
import sys
sys.path.insert(0, 'functions/'); # add this path for makeStimulus
import makeStimulus
import helper_fcns as hf
import autoreload

import pdb

# constants - directories
base_loc = '/arc/2.2/p1/plevy/SF_diversity/sfDiv-OriModel/sfDiv-python/';

loc_matData = 'V1/structures/'; # where are the .mat files?
loc_pyData = 'V1/structures/'; # where do you want the .npy files?

## The original conversion

### Converting files

First, get the .mat files to convert; then, convert unless already done 

In [2]:
files = os.listdir(loc_matData)

In [3]:
for i in files:
    if i.find('#') >= 0:
#         os.rename(loc_matData + i, loc_matData + i.replace('#', ''))
        print('renaming %s to %s' % (loc_matData + i, loc_matData + i.replace('#', '')))


In [8]:
# convert individual files
unitName = [];
for i in files:
    # if file has 'sfm' in it and starts with m then 
    if i.find('sfm') >= 0 and i.startswith('m'):
        
        don't convert if it already exists
        if os.path.exists(loc_pyData + i.replace('.mat', '.npy')):
            if i.endswith('.npy'): # only add once (not also with .mat)
                unitName.append(i[0:i.find('_')]) # go up to the '_' character
            continue;
                
        print("loading: " + i)
        matData = makeStimulus.loadmat(loc_matData + i);
        S = matData.get('S'); # the actual data structure
        
        print("now saving...")
        saveName = loc_pyData + i.replace('.mat', '.npy');
        np.save(saveName, S)
        
        unitName.append(i[0:i.find('_')]) # go up to the '_' character

loading: m676l1_sfm.mat
now saving...


In [7]:
unitName

[]

Create/update data list

In [10]:
if os.path.exists(loc_pyData + 'dataList.npy'):
    dataList = np.load(loc_pyData + 'dataList.npy').item();
    dataList['unitName'] = unitName;
    np.save(loc_pyData + 'dataList.npy', dataList);
else: # unitType, isolation, comment must be filled in by hand at later time
    dataList = dict();
    dataList['unitName'] = unitName;
    dataList['unitType'] = [];
    dataList['isolation'] = [];
    dataList['comment'] = [];
    np.save(loc_pyData + 'dataList.npy', dataList);

In [11]:
dataList = np.load(loc_pyData + 'dataList.npy').item();

In [12]:
dataList

{'comment': [],
 'isolation': [],
 'unitName': ['m676l1', 'm676p3l6', 'm676p3l13', 'm676p3l15', 'm676p3l7'],
 'unitType': []}

Likely not needed, this section was from converting the previous data set where normalization responses where already computed in Matlab

In [ ]:
# change some things around...
for i in dataList['unitName']:

    print("changing: " + i)
    S = np.load(loc_pyData + i + '_sfm.npy').item(); # the actual data structure
    
    if S.get('sfm').get('mod'):
        if S.get('sfm').get('mod').get('normalization') and S.get('sfm').get('mod').get('normalization_py'):
            
            S['sfm']['mod']['norm_old'] = S['sfm']['mod']['normalization'];
            S['sfm']['mod']['normalization'] = S['sfm']['mod']['normalization_py'];
            S['sfm']['mod'].pop('normalization_py');
            
            print("now saving...")
            saveName = loc_pyData + i + '_sfm.npy';
            np.save(saveName, S)

## General 'update python structs' here

Use this section to update something about each cell in the dataList (change appropriate field(s))

In [24]:
dL = np.load(loc_pyData + 'dataList.npy').item();

In [32]:
for i in dL['unitName']:
    S = np.load(loc_pyData + i + '_sfm.npy').item();
    if 'descr_model' in S['sfm']:
        S['sfm'].pop('descr_model');
        np.save(loc_pyData + i + '_sfm.npy', S);
        print('Removed descr_model from ' + i);